In [ ]:
filename = "1_100.log"

In [ ]:
from pathlib import Path

if not Path(filename).exists():
    !grep "Forwarding request " ../logs/receiver*.log >reordering.log

In [ ]:
import re
def parse_int_tag(line, name):
    match = re.search(f"{name}=([0-9]*)", line)
    if match:
        return int(match.group(1))
    else:
        return 0

def parse_line(line):
    deadline = parse_int_tag(line, "deadline")
    now = parse_int_tag(line, "now")
    id = parse_int_tag(line, "r_id")
    proxy_id = parse_int_tag(line, "c_id")
    proxy_seq = parse_int_tag(line, "c_seq")

    return (id, proxy_id, proxy_seq, now - deadline)    

In [ ]:
requests = {}

for line in open(filename):
    (id, *req) = parse_line(line)

    if id not in requests:
        requests[id] = []
        
    requests[id].append(tuple(req))

In [ ]:
# Get edit distance between 2 sequences from receivers
def req_eq(r1, r2):
    return r1[0] == r2[0] and r1[1] == r2[1]

def edit_dist(s1, s2):
    dp = [ ([-1] * (len(s1) + 1)) for _ in range(len(s2) + 1)]

    dp[0][0] = 0

    for i in range(1, len(dp)):
        dp[i][0] = i
        
    for j in range(1, len(dp[0])):
        dp[0][j] = j
        
    for i in range(1, len(dp)):
        for j in range(1, len(dp[i])):
            # Change req at position i/j
            if req_eq(s1[i-1], s2[i-1]):
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + dp[i - 1][j - 1]

            # TODO swaps?
            dp[i][j] = min(dp[i][j], 1 + dp[i - 1][j], 1 + dp[i][j - 1])
    
    return dp[-1][-1]

# Simpler metric, which perecentage of requests are the same?
def matching_reqs(*seqs):
    n = len(seqs[0])
    ret = []
    
    for entries in zip(*seqs):
        match = True
        for (x, y) in zip(entries, entries[1:]):
            if not req_eq(x, y): match = False

        ret.append(match)
        
    return ret

In [ ]:
import itertools

reqs = matching_reqs(*[v for v in requests.values()])

print(f"Total number of requests: {len(sections)}")
print(f"Total proportion of requests the same: {sum(sections) / len(sections):.2f}")

# Get runs of smae requests len 10 or above
print([(k, sum(1 for _ in g)) for k, g in itertools.groupby(reqs)])

In [ ]:
print(edit_dist(requests[0], requests[1]))
print(edit_dist(requests[0], requests[2]))
print(edit_dist(requests[0], requests[3]))